#Importing the necessary libraries

In [1]:
import cv2
import os
import numpy as np
import sys
import glob
import time
import torch
from google.colab import files
from google.colab.patches import cv2_imshow
from IPython import display

#Copy the project repository:

In [2]:
!git clone 'https://github.com/SavasAtt/Detect-Track-and-count-objects-in-live-videos-using-YOLOv5-and-ByteTracker'

Cloning into 'Detect-Track-and-count-objects-in-live-videos-using-YOLOv5-and-ByteTracker'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), 37.81 MiB | 8.99 MiB/s, done.


#Copy YOLOv5 repository and install the necessary libraries:

In [3]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

%cd yolov5/
!pwd
!pip install -r requirements.txt
%cd ..

!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

display.clear_output()

# Install all necessary libraries and dependencies for TensorRT:

install these libraries only if you going to use TensorRT because it wil take long time to be installed.

In [4]:
!pip install nvidia-pyindex
!pip install nvidia-tensorrt
!pip3 install tensorrt-5.0.2.6-py2.py3-none-any.whl  
!pip3 install pycuda 

display.clear_output()

# Copy the trained **YOLOv5s** model and convert it to **ONNX** and **TensorRT**:

In [5]:
import shutil

shutil.copyfile('/content/Detect-Track-and-count-objects-in-live-videos-using-YOLOv5-and-ByteTracker/best_model_YOLOv5s.pt', 
                '/content/best_model_YOLOv5s.pt')

!`python /content/yolov5/export.py --weights /content/best_model_YOLOv5s.pt --include torchscript engine --device 0`

export: data=yolov5/data/coco128.yaml, weights=['/content/best_model_YOLOv5s.pt'], imgsz=[640, 640], batch_size=1, device=0, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'engine']
YOLOv5 🚀 v7.0-97-gfa4bdbe Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /content/best_model_YOLOv5s.pt with output shape (1, 25200, 7) (13.8 MB)

TorchScript: starting export with torch 1.13.1+cu116...
TorchScript: export success ✅ 1.5s, saved as /content/best_model_YOLOv5s.torchscript (27.2 MB)
requirements: YOLOv5 requirement "onnx>=1.12.0" not found, attempting AutoUpdate...
ERROR: pip's dependency resolver does not currently take into account all the packages that are

## Install ByteTrack

[ByteTrack](https://github.com/ifzhang/ByteTrack) is great tracker and we can use it with [YOLOv5].

In [6]:
HOME = '/content'

%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack && pip3 install -q -r requirements.txt
!cd ByteTrack && python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric

,
display.clear_output()

import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [7]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

## Install Roboflow Supervision

In [8]:
!pip install -i https://test.pypi.org/simple/ supervision


display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.2.0


In [9]:
from supervision.draw.color import ColorPalette
from supervision.geometry.utils import Point
from supervision import VideoInfo
from supervision import get_video_frames_generator
from supervision import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision import Detections, BoxAnnotator
# from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

# Tracking utils and functions:

In [10]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections, 
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)
    
    tracker_ids = [None] * len(detections)
    
    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

#Copy test video and make second copy of it:

In [12]:
import shutil

src="/content/Detect-Track-and-count-objects-in-live-videos-using-YOLOv5-and-ByteTracker/test.mp4"
dst="/content/Detect-Track-and-count-objects-in-live-videos-using-YOLOv5-and-ByteTracker/test2.mp4"

shutil.copy(src,dst)

'/content/Detect-Track-and-count-objects-in-live-videos-using-YOLOv5-and-ByteTracker/test2.mp4'

In [13]:
# settings
# LINE_START = Point(0, 50)
# LINE_END = Point(640, 50)
%cd {HOME}
TARGET_VIDEO_PATH = src
TARGET_VIDEO_PATH2 = dst
VideoInfo.from_video_path(TARGET_VIDEO_PATH)

/content


VideoInfo(width=640, height=640, fps=30, total_frames=1800)

#Setting parameters for ByteTracker:

In [14]:
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.7
    track_buffer: int = 100
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 10
    mot20: bool = False

# Setting Model format to be **'PT'** or **'ONNX'** or **'TensorRT'**:

In [15]:
model_type = 'engine' # 'pt' or 'onnx' or 'engine'
best_model_path = f'/content/best_model_YOLOv5s.{model_type}'
model = torch.hub.load('ultralytics/yolov5', 'custom', best_model_path)
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

/usr/local/lib/python3.8/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-2-12 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Loading /content/best_model_YOLOv5s.engine for TensorRT inference...


[Errno 2] No such file or directory: '/usr/local/lib/python3.8/dist-packages/setuptools-57.4.0.dist-info/METADATA'


Adding AutoShape... 


#Create **classes name dictionary** and class **IDs**:

In [16]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = {0: 'Bolt', 1: 'Nut'}
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [0, 1]

#Process the test video:

Here the test video is processed and all required operations are done **(Detecting, tracking and counting)**. Also **live FPS, mean FPS and max FPS** is calculated and displayed on the video.
finally the processed video is dowonloaded.

In [17]:
# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(TARGET_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(TARGET_VIDEO_PATH)
# # create LineCounter instance
# line_counter = LineCounter(start=LINE_START, end=LINE_END)
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(thickness=4, text_thickness=1, text_scale=0.6)
# line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)

FPS_list = []
if model_type == 'pt':
  model_format = 'PT'
elif model_type == 'onnx':
  model_format = 'ONNX'
elif model_type == 'engine':
  model_format = 'TensorRT'

Sink_container = VideoSink(TARGET_VIDEO_PATH2,video_info)
Sink_container.target_path = '/content/output_' + model_format + '.mp4'

# open target video file
with Sink_container as sink:
    # loop over video frames
    for frame in generator:
        # model prediction on single frame and conversion to supervision Detections

        start = time.perf_counter()
        results = model(frame)
        detections = Detections(
            xyxy=results.xyxyn[0][:, :-2].cpu().numpy() * video_info.width,
            confidence=results.xyxyn[0][:, 4].cpu().numpy(),
            class_id=results.xyxyn[0][:, -1].cpu().numpy().astype(int)
        )
        # print(detections)
        print('')
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # filtering out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # format custom labels
        labels = [
            f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        # # updating line counter
        # line_counter.update(detections=detections)
        # annotate and display frame
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
        
        end = time.perf_counter()
        totalTime = end - start
        fps = 1 / totalTime
        FPS_list.append(fps)
        min_fps = min(FPS_list)
        max_fps = max(FPS_list)
        mean_fps = round(sum(FPS_list)/len(FPS_list))

        cv2.putText(frame, f'YOLO5s[{model_format}]', (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f'FPS: {int(fps)}', (20,60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv2.putText(frame, f'Mean FPS: {int(mean_fps)}', (20,90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)
        cv2.putText(frame, f'Max FPS: {int(max_fps)}', (20,120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        cv2.putText(frame, 'GPU: ' + torch.cuda.get_device_name(0), (20,150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)

        sink.write_frame(frame)
# display.clear_output()

print('Video process is done.')
print('Model: YOLO5s.' + model_format)
print('GPU: ' + torch.cuda.get_device_name(0))
print('Mean FPS:' + str(mean_fps))
print('Max FPS:' + str(max_fps))
print('')
print('Now downloading the processed video:')
files.download('/content/output_' + model_format + '.mp4') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>